# 구글 플레이 스토어 리뷰 웹 크롤러
- source : https://github.com/park-gb/playstore-review-crawler

# 초기 설정

####  크롬(Chrome)  브라우저 버전과 비슷한 chromedriver 프로그램 다운로드가 필요합니다.
https://chromedriver.chromium.org/downloads

In [6]:
chrome_driver = './chromedriver' # 파일 확장자 제외

## 앱이름 및 스크롤 카운트 설정

In [7]:
APP_NAME = 'com.ktmusic.geniemusic'
# APP_NAME = 'com.iloen.melon'
# APP_NAME = 'com.google.android.apps.youtube.music'
SCROLL_COUNT = 50

## 수집 앱 URL

In [8]:
URL = f'https://play.google.com/store/apps/details?id={APP_NAME}'

# 패키지 import

In [11]:
!pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org selenium
!pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org tqdm

You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.


In [12]:
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from time import sleep
import random
from tqdm.auto import tqdm, trange
import pandas as pd

# 스크롤 함수

In [15]:
def scroll(modal):
    try:        
        # 스크롤 높이 받아오기
        last_height = driver.execute_script("return arguments[0].scrollHeight", modal)
#         while True:
        for i in range(SCROLL_COUNT):
            pause_time = random.uniform(0.5, 0.8)
            # 최하단까지 스크롤
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", modal)
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight-50);", modal)
            time.sleep(pause_time)
            # 스크롤 높이 새롭게 받아오기
            new_height = driver.execute_script("return arguments[0].scrollHeight", modal)
            try:
                # '더보기' 버튼 있을 경우 클릭
                all_review_button = driver.find_element_by_xpath('/html/body/div[1]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div[2]/div/span/span').click()
            except:
                # 스크롤 완료 경우
                if new_height == last_height:
                    print("스크롤 완료")
                    break
                last_height = new_height
                
    except Exception as e:
        print("에러 발생: ", e)

# 데이터 크롤링

In [16]:
# 크롬 드라이버 세팅
driver = webdriver.Chrome(chrome_driver)
# 페이지 열기
driver.get(URL)
# 페이지 로딩 대기
wait = WebDriverWait(driver, 5)

<ipython-input-16-d366c92a2943>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver)


In [19]:
# '리뷰 모두 보기' 버튼 렌더링 확인
all_review_button_xpath = '//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[1]/div[2]/div/div[1]/c-wiz[4]/section/div/div/div[5]/div/div/button/span'
button_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, all_review_button_xpath)))

# '리뷰 모두 보기' 버튼 클릭
# review_btn = driver.find_element_by_xpath(all_review_button_xpath)
review_btn = driver.find_element("xpath", all_review_button_xpath)
review_btn.click()

In [20]:
# '리뷰 모두 보기' 페이지 렌더링 대기
all_review_page_xpath = '/html/body/div[4]/div[2]/div/div/div/div/div[2]'
page_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, all_review_page_xpath)))

In [21]:
# 페이지 스크롤 다운
modal = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='fysCi']")))
scroll(modal)

KeyboardInterrupt: 

# HTML Parsing

In [22]:
# html parsing하기
html_source = driver.page_source
soup_source = BeautifulSoup(html_source, 'html.parser')

# HTML 데이터 저장

In [23]:
# html 데이터 저장
with open(f"./data_{APP_NAME}.html", "w", encoding = 'utf-8') as file:
    file.write(str(soup_source))

## 데이터프레임 변환

In [24]:
# 리뷰 데이터 클래스 접근
review_source = soup_source.find_all(class_ = 'RHo1pe')
# 리뷰 데이터 저장용 배열
dataset = []
# 데이터 넘버링을 위한 변수
review_num = 0 

# 리뷰 1개씩 접근해 정보 추출
for review in tqdm(review_source):
    review_num+=1
    # 리뷰 등록일 데이터 추출
    date_full = review.find_all(class_ = 'bp9Aid')[0].text
    date_year = date_full[0:4] # 연도 데이터 추출
    # 해당 단어가 등장한 인덱스 추출
    year_index = date_full.find('년')
    month_index = date_full.find('월')
    day_index = date_full.find('일')
    
    date_month = str(int(date_full[year_index+1:month_index])) # 월(Month) 데이터 추출
    # 월 정보가 1자리의 경우 앞에 0 붙이기(e.g., 1월 -> 01월)
    if len(date_month) == 1:
        date_month = '0' + date_month
    
    date_day = str(int(date_full[month_index+1:day_index])) # 일(Day) 데이터 추출 
    # 일 정보가 1자리의 경우 앞에 0 붙여줌(e.g., 7일 -> 07일)
    if len(date_day) == 1:
        date_day = '0' + date_day
    
    # 리뷰 등록일 full version은 최종적으로 yyyymmdd 형태로 저장
    date_full = date_year + date_month + date_day
    user_name = review.find_all(class_ = 'X5PpBb')[0].text # 닉네임 데이터 추출
    rating = review.find_all(class_ = "iXRFPc")[0]['aria-label'][10] # 평점 데이터 추출
    content = review.find_all(class_ = 'h3YV2d')[0].text # 리뷰 데이터 추출

    data = {
        "id": review_num, 
        "date": date_full,
        "dateYear": date_year,
        "dateMonth": date_month,
        "dateDay": date_day,
        "rating": rating,
        "userName": user_name,
        "content": content
    }
    dataset.append(data)

  0%|          | 0/320 [00:00<?, ?it/s]

## 데이터프레임 저장

In [25]:
df = pd.DataFrame(dataset)
df.to_csv(f'./review_dataset_{APP_NAME}.csv', encoding = 'utf-8-sig') # csv 파일로 저장

# 리뷰 데이터 불러오기

In [26]:
# 저장한 리뷰 정보 불러오기
df = pd.read_csv(f'./review_dataset_{APP_NAME}.csv', encoding = 'utf-8-sig')
df = df.drop(['Unnamed: 0'], axis = 1) # 불필요한 칼럼 삭제
df

,id,date,dateYear,dateMonth,dateDay,rating,userName,content
0,1,20220607,2022,6,7,4,이종현,특정 아티스트 제외 옵션 좀 넣어주세요. 플레이리스트를 매번 만들수도 없고. 신곡 ...
1,2,20220619,2022,6,19,2,코바,제발 음질개선좀 신경써주세요.... 통신사할인으로 지니뮤직을 쓰고있긴한데 진짜 음질...
2,3,20220610,2022,6,10,3,김성배,재생목록 및 보관함음악등이 한개씩밖에 등록이 안되며 이동시 기존들었던 내역이 기록되...
3,4,20220616,2022,6,16,5,jim jjang,사용한지 1년 지금까지 아무곳에서도 사용하지않는 특별한 ui/ux로 사람을 혼란시킴...
4,5,20220617,2022,6,17,1,김나연,"오류가 너무 많이 나고, 재생목록도 제가 넣은 순서가 아니라 순서도 다르게 지정되네..."
...,...,...,...,...,...,...,...,...
315,316,20190221,2019,2,21,4,Google 사용자,제가 지니계정과 카카오톡 연동 계정이 있는데 둘을 합치거나 할 수 있으면 좋겠네요 ...
316,317,20200211,2020,2,11,1,Google 사용자,업데이트후 잦은 오류 너무 불편해요 노래가 갑자기 다음으로 넘어가지고 다음노래로 넘...
317,318,20210123,2021,1,23,1,ASTERISM,"음악재생시 상단바에 나오는 음악제목, 앨범아트가 안 바뀝니다. 기기를 껐다켜고 다시..."
318,319,20200818,2020,8,18,2,엄준현,1. 한달에 한두번쯤 스트리밍 및 로그인 되어있는데도 불구하고 1분30초 무료듣기라...


### 감사합니다.